In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Printf
using JLD2

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\HW1\Project.toml`


# Q2 (30 pts): Augmented Lagrangian Quadratic Program Solver

## Part (A): QP Solver (10 pts)
Here we are going to use the augmented lagrangian method described [here in a video](https://www.youtube.com/watch?v=0x0JD5uO_ZQ), with [the corresponding pdf here](https://github.com/Optimal-Control-16-745/lecture-notebooks-2022/blob/main/misc/AL_tutorial.pdf) to solve the following problem:

$$\begin{align}
\min_x \quad & \frac{1}{2}x^TQx + q^Tx \\ 
\mbox{s.t.}\quad &  Ax -b = 0 \\ 
&  Gx - h \leq 0 
\end{align}$$
where the cost function is described by $Q \in \mathbb{R}^{n \times n}$, $q \in \mathbb{R}^n$, an equality constraint is described by $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$, and an inequality constraint is described by $G \in \mathbb{R}^{p \times n}$ and $h \in \mathbb{R}^p$.


By introducing a dual variable $\lambda \in \mathbb{R}^m$ for the equality constraint, and $\mu \in \mathbb{R}^p$ for the inequality constraint, we have the following KKT conditions for optimality:

$$\begin{align}
Qx + q + A^T\lambda + G^T \mu &= 0 \quad \quad \text{stationarity}\\ 
Ax-b&= 0 \quad \quad \text{primal feasibility} \\ 
Gx-h&\leq 0 \quad \quad \text{primal feasibility} \\ 
\mu &\geq 0 \quad \quad \text{dual feasibility} \\ 
\mu \circ (Gx - h) &= 0 \quad \quad \text{complementarity}
  \end{align}$$
> where $\circ$ is element-wise multiplication.  

In [15]:
function h_ineq(x::Vector)::Vector
    G = 0.1*I(size(x,1))
    h = 0.2*ones(size(x))
    @show G h

    return G*x - h
end

x = [1, 2]
x_res = h_ineq(x)
@show x_res size(x_res)

for i in 1:size(x_res, 1)
    @show x_res[i]
end

I_temp = 0.1 * I(size(x_res, 1))
I_temp[1,1]
norm(x_res)

G = [0.1 0.0; 0.0 0.1]
h = [0.2, 0.2]
x_res = [-0.1, 0.0]
size(x_res) = (2,)
x_res[i] = -0.1
x_res[i] = 0.0


0.1

In [21]:
# TODO: read below
# NOTE: DO NOT USE A WHILE LOOP ANYWHERE
"""
The data for the QP is stored in `qp` the following way:
    @load joinpath(@__DIR__, "qp_data.jld2") qp 

which is a NamedTuple, where
    Q, q, A, b, G, h = qp.Q, qp.q, qp.A, qp.b, qp.G, qp.h

contains all of the problem data you will need for the QP.

Your job is to make the following function 
    
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)

You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:

as long as solve_qp works. 
"""
function cost(qp::NamedTuple, x::Vector)::Real
    0.5*x'*qp.Q*x + dot(qp.q,x)
end
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end

function kkt_conditions(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector)::Vector

    stationary = qp.Q*x + qp.q + qp.A'*λ + qp.G'*μ
    primal = c_eq(qp, x)
    complementary = h_ineq(qp, x) .* μ
    
    return [
        stationary;
        primal;
        complementary;
    ]
end
        
function mask_matrix(qp::NamedTuple, x::Vector, μ::Vector, ρ::Real)::Matrix

    H = h_ineq(qp, x)
    Iρ = zeros(size(μ, 1), size(μ, 1))
    
    for i in 1:size(μ, 1)
        if ((H[i] < 0) && (μ[i] == 0))
            Iρ[i,i] = 0
        else
            Iρ[i,i] = ρ
        end
    end

    return Iρ
end

function augmented_lagrangian(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector, ρ::Real)::Real
    
    C = c_eq(qp, x)
    H = h_ineq(qp, x)
    Iρ = mask_matrix(qp, x, μ, ρ)

    L = cost(qp, x) + λ'*C + μ'*H + 0.5*ρ*C'*C + 0.5*H'*Iρ*H

    return L
end

function logging(qp::NamedTuple, main_iter::Int, AL_gradient::Vector, x::Vector, λ::Vector, μ::Vector, ρ::Real)
    # TODO: stationarity norm
    # ∇L = FD.gradient(_x -> augmented_lagrangian(qp, _x, λ, μ, ρ), x)
    ∇L = qp.Q*x + qp.q + qp.A'*λ + qp.G'*μ
    stationarity_norm = norm(∇L)
    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e\n",
          main_iter, stationarity_norm, norm(AL_gradient), maximum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(μ,h_ineq(qp,x))), ρ)
end
function solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)
    x = zeros(length(qp.q))
    λ = zeros(length(qp.b))
    μ = zeros(length(qp.h))
    ρ = 1.0
    ϕ = 10.0
    
    if verbose
        @printf "iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ\n"
        @printf "----------------------------------------------------------------\n"
    end
    
    for main_iter = 1:max_iters 
        # NOTE: when you do your dual update for μ, you should compute
        # your element-wise maximum with `max.(a,b)`, not `max(a,b)`
        
        ∇L = FD.gradient(_x -> augmented_lagrangian(qp, _x, λ, μ, ρ), x)
        if norm(∇L) > 1e-6
            ∇²L = FD.hessian(_x -> augmented_lagrangian(qp, _x, λ, μ, ρ), x)
            x = x - ∇²L\∇L
        else
            λ = λ + ρ*c_eq(qp, x)
            μ = max.(0, μ+ρ*h_ineq(qp, x))
            ρ = ρ * ϕ
        end
        
        if verbose
            logging(qp, main_iter, ∇L, x, λ, μ, ρ)
        end

        # TODO: convergence criteria based on tol 
        if norm(kkt_conditions(qp, x, λ, μ)) < tol
            return x, λ, μ
        end
    end
    error("qp solver did not converge")
end

let 
    # example solving qp 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, tol = 1e-8)
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   4.46e+00   5.60e+01   1.55e+00   1.31e+00   0.00e+00  1e+00
  2   4.92e+00   4.83e+00   5.51e-01   1.27e+00   0.00e+00  1e+00
  3   4.52e-15   4.88e-15   5.51e-01   1.27e+00   4.59e-01  1e+01
  4   4.97e+00   4.92e+01   2.56e-02   3.07e-01   4.94e-02  1e+01
  5   6.16e+00   5.55e-14   2.56e-02   3.07e-01   1.05e-02  1e+02
  6   4.79e+00   8.87e+01   6.84e-03   1.35e-02   4.55e-04  1e+02
  7   5.52e-01   3.75e-13   6.84e-03   1.35e-02   7.94e-03  1e+03
  8   5.38e-01   4.28e+01   6.84e-02   1.55e-04   1.40e-04  1e+03
  9   5.30e-01   2.13e+02   3.64e-05   1.62e-04   1.17e-04  1e+03
 10   4.05e-12   4.05e-12   3.64e-05   1.62e-04   1.06e-04  1e+04
 11   5.21e-04   5.30e+00  -5.61e-09   2.05e-08   1.14e-08  1e+04
 12   5.02e-11   5.02e-11  -5.61e-09   2.05e-08   1.14e-08  1e+05
 13   1.04e-07   5.21e-03  -1.58e-13   3.92e-13   1.88e-13  1e+05
 14   4.51e-10

([-0.3262308057134027, 0.24943797997188863, -0.4322676644050709, -1.4172246971241347, -1.3994527400876544, 0.6099582408523683, -0.07312202122159435, 1.3031477521999633, 0.53890347910655, -0.7225813651685942], [-0.12835195123665755, -2.8376241673439813, -0.832080449930223], [0.036352942606843186, 0.0, 0.0, 1.0594444951590334, 0.0])

### QP Solver test

In [22]:
# 10 points 
using Test 
@testset "qp solver" begin 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-6)
    
    @load joinpath(@__DIR__, "qp_solutions.jld2") qp_solutions
    @test norm(x - qp_solutions.x,Inf)<1e-3;
    @test norm(λ - qp_solutions.λ,Inf)<1e-3;
    @test norm(μ - qp_solutions.μ,Inf)<1e-3;
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   4.46e+00   5.60e+01   1.55e+00   1.31e+00   0.00e+00  1e+00
  2   4.92e+00   4.83e+00   5.51e-01   1.27e+00   0.00e+00  1e+00
  3   4.52e-15   4.88e-15   5.51e-01   1.27e+00   4.59e-01  1e+01
  4   4.97e+00   4.92e+01   2.56e-02   3.07e-01   4.94e-02  1e+01
  5   6.16e+00   5.55e-14   2.56e-02   3.07e-01   1.05e-02  1e+02
  6   4.79e+00   8.87e+01   6.84e-03   1.35e-02   4.55e-04  1e+02
  7   5.52e-01   3.75e-13   6.84e-03   1.35e-02   7.94e-03  1e+03
  8   5.38e-01   4.28e+01   6.84e-02   1.55e-04   1.40e-04  1e+03
  9   5.30e-01   2.13e+02   3.64e-05   1.62e-04   1.17e-04  1e+03
 10   4.05e-12   4.05e-12   3.64e-05   1.62e-04   1.06e-04  1e+04
 11   5.21e-04   5.30e+00  -5.61e-09   2.05e-08   1.14e-08  1e+04
 12   5.02e-11   5.02e-11  -5.61e-09   2.05e-08   1.14e-08  1e+05
Test Summary: | Pass  Total
qp solver     |    3      3


Test.DefaultTestSet("qp solver", Any[], 3, false, false)

# Simulating a Falling Brick with QPs
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \mu \\ \text{ where } M = mI_{2\times 2}, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\mu \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ \Delta t \cdot \begin{bmatrix} \frac{1}{m} J^T \mu_{k+1} - g \\ v_{k+1} \end{bmatrix}$$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\mu_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\mu_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (B): QP formulation for Falling Brick (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + [M (\Delta t \cdot g - v_k)]^Tv_{k+1} \\
    &\text{subject to} && -J(q_k + \Delta t \cdot v_{k+1}) \leq 0 \\
\end{align} $$

**TASK**: Write down the KKT conditions for the optimization problem above, and show that it's equivalent to the dynamics problem stated previously. Use LaTeX markdown.

**PUT ANSWER HERE:**

## Part (C): Brick Simulation (5 pts)

In [ ]:
function brick_simulation_qp(q, v; mass = 1.0, Δt = 0.01)
    
    # TODO: fill in the QP problem data for a simulation step 
    # fill in Q, q, G, h, but leave A, b the same 
    # this is because there are no equality constraints in this qp 
    
    qp = (
        Q = zeros(2,2), 
        q = zeros(2),
        A = zeros(0,2), # don't edit this
        b = zeros(0),   # don't edit this 
        G = zeros(1,2),
        h = zeros(1)
    )
    
    return qp 
end

In [ ]:
@testset "brick qp" begin 
    
    q = [1,3.0]
    v = [2,-3.0]
    
    qp = brick_simulation_qp(q,v)
    
    # check all the types to make sure they're right
    qp.Q::Matrix{Float64}
    qp.q::Vector{Float64}
    qp.A::Matrix{Float64}
    qp.b::Vector{Float64}
    qp.G::Matrix{Float64}
    qp.h::Vector{Float64}
    
    @test size(qp.Q) == (2,2)
    @test size(qp.q) == (2,)
    @test size(qp.A) == (0,2)
    @test size(qp.b) == (0,)
    @test size(qp.G) == (1,2)
    @test size(qp.h) == (1,)
    
    @test abs(tr(qp.Q) - 2) < 1e-10
    @test norm(qp.q - [-2.0, 3.0981]) < 1e-10 
    @test norm(qp.G - [0 -.01]) < 1e-10 
    @test abs(qp.h[1] -3) < 1e-10
    
end

In [ ]:
include(joinpath(@__DIR__, "animate_brick.jl"))
let 
    
    dt = 0.01 
    T = 3.0 
    
    t_vec = 0:dt:T
    N = length(t_vec)
    
    qs = [zeros(2) for i = 1:N]
    vs = [zeros(2) for i = 1:N]
    
    qs[1] = [0, 1.0]
    vs[1] = [1, 4.5]
    
    # TODO: simulate the brick by forming and solving a qp 
    # at each timestep. Your QP should solve for vs[k+1], and
    # you should use this to update qs[k+1]

    
    xs = [q[1] for q in qs]
    ys = [q[2] for q in qs]
    
    @show @test abs(maximum(ys)-2)<1e-1
    @show @test minimum(ys) > -1e-2
    @show @test abs(xs[end] - 3) < 1e-2
    
    xdot = diff(xs)/dt
    @show @test maximum(xdot) < 1.0001
    @show @test minimum(xdot) > 0.9999
    @show @test ys[110] > 1e-2
    @show @test abs(ys[111]) < 1e-2
    @show @test abs(ys[112]) < 1e-2
    
    display(plot(xs, ys, ylabel = "y (m)", xlabel = "x (m)"))
    
    animate_brick(qs)
    
    
    
end

# Part D (5 pts): Solve a QP

Use your QP solver to solve the following optimization problem:


$$
\begin{align} 
\min_{y\in\mathbb{R}^2,a\in\mathbb{R},b\in\mathbb{R}} \quad & \frac{1}{2}y^T \begin{bmatrix} 1 & .3 \\ .3 & 1 \end{bmatrix} y + a^2 + 2b^2  + \begin{bmatrix} -2 & 3.4 \end{bmatrix} y + 2a + 4b \\ 
\text{st} \quad & a + b = 1 \\ 
& \begin{bmatrix}-1 & 2.3 \end{bmatrix} y + a - 2b =3 \\
& -0.5 \leq y \leq 1 \\ 
& -1 \leq a \leq 1 \\ 
& -1 \leq b \leq 1
\end{align}
$$

You should be able to put this into our standard QP form that we used above, and solve.

In [ ]:
@testset "part D" begin

    y = randn(2)
    a = randn()
    b = randn()
    
    
    @test norm(y - [-0.080823; 0.834424]) < 1e-3 
    @test abs(a - 1) < 1e-3 
    @test abs(b) < 1e-3 
end

## Part E (5 pts): One sentence short answer

1. For our Augmented Lagrangian solver, if our initial guess for $x$ is feasible (meaning it satisfies the constraints), will it stay feasible through each iteration? 

**put ONE SENTENCE answer here**

2. Does the Augmented Lagrangian function for this problem always have continuous first derivatives?

**put ONE SENTENCE answer here** 

3. Is the QP in part D always convex?

**put ONE SENTENCE answer here**